In [18]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
pd.set_option("display.max_columns", None)


#load the data
data = pd.read_csv("Flight2019.csv")
#airlines = pd.read_csv("~/Documents/PycharmProjects/Flights/venv/airlines.csv")
#airports = pd.read_csv("~/Documents/PycharmProjects/Flights/venv/airports.csv")



In [17]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [19]:
def clean_variables_1(data):
    
    """
    Clean variables that provide no information for delays
    """
    
    NAs_to_remove = ["Quarter","Flight_Number_Reporting_Airline", "Div5TailNum", "Div5WheelsOff", "Div5LongestGTime",
                        "Div5LongestGTime", "Div5TotalGTime", "Div5WheelsOn", "Div5AirportSeqID",
                        "Div5AirportID", "Div5Airport", "Div4TailNum", "Div4WheelsOff", "Div4LongestGTime",
                        "Div4TotalGTime", "Div4WheelsOn", "Div4AirportSeqID", "Div4AirportID", "Div4Airport",
                        "Div3TailNum", "Div3WheelsOff", "Div3LongestGTime", "Div3TotalGTime", "Div3WheelsOn",
                        "Div3AirportSeqID","Div3AirportID", "Div3Airport", "Div2TailNum", "Div2WheelsOff",
                        "Div2LongestGTime", "Div2TotalGTime", "Div2WheelsOn", "Div2AirportSeqID", "Div2AirportID",
                        "Div2Airport", "Div1TailNum", "Div1WheelsOff", "Div1LongestGTime", "Div1TotalGTime",
                        "Div1WheelsOn","Div1AirportSeqID", "Div1AirportID", "Div1Airport", "DivDistance",
                        "DivArrDelay", "DivActualElapsedTime","DivReachedDest", "DivAirportLandings",
                        "LongestAddGTime", "DistanceGroup", "CancellationCode", "DestState", "DestStateFips",
                        "DestStateName", "DestWac", "OriginWac", "OriginStateName", "OriginStateFips",
                        "OriginState", "FirstDepTime", "TotalAddGTime", "Cancelled","Unnamed: 109","OriginCityName","ArrDelay"]
    
    data = data.drop(NAs_to_remove, axis = 1)
    
    return data


def clean_variables_2(data):
    
    """
    Clean variables that do provide input but can have multicollinearity issues
    """
    
    other_columns = ["Year", "Month", "DayofMonth", "FlightDate", "Reporting_Airline", "Tail_Number", "DOT_ID_Reporting_Airline",
                    "OriginAirportID", "OriginAirportSeqID", "OriginCityMarketID",
                    "DestAirportID", "DestAirportSeqID", "DestCityMarketID", "DestCityName",
                    "DepDelay", "DepartureDelayGroups", "DepTimeBlk", "ArrTime", "ArrivalDelayGroups", "ArrTimeBlk",
                    "Diverted", "AirTime", "Flights"]
    
    data = data.drop(other_columns, axis = 1)
    
    return data


def clean_variables_3(data):
    
    """
    Clean delay variables
    """
    
    pot_vars = ["CarrierDelay", "WeatherDelay", "NASDelay", "SecurityDelay", "LateAircraftDelay"]
    
    data = data.drop(pot_vars, axis = 1)
    
    return data

data = clean_variables_1(data)
data = clean_variables_2(data)
data = clean_variables_3(data)
data.head()

,DayOfWeek,IATA_CODE_Reporting_Airline,Flight_Number_Reporting_Airline,Origin,OriginCityName,Dest,CRSDepTime,DepTime,DepDelayMinutes,DepDel15,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,CRSElapsedTime,ActualElapsedTime,Distance
0,6,OH,5213,OAJ,"Jacksonville/Camp Lejeune, NC",CLT,1036,1032.0,0.0,0.0,9.0,1041.0,1119.0,8.0,1201,-34.0,0.0,0.0,85.0,55.0,191.0
1,7,OH,5213,OAJ,"Jacksonville/Camp Lejeune, NC",CLT,1036,1054.0,18.0,1.0,13.0,1107.0,1147.0,7.0,1201,-7.0,0.0,0.0,85.0,60.0,191.0
2,1,OH,5213,OAJ,"Jacksonville/Camp Lejeune, NC",CLT,1036,1112.0,36.0,1.0,12.0,1124.0,1209.0,4.0,1201,12.0,12.0,0.0,85.0,61.0,191.0
3,2,OH,5213,OAJ,"Jacksonville/Camp Lejeune, NC",CLT,1036,1112.0,36.0,1.0,15.0,1127.0,1211.0,13.0,1201,23.0,23.0,1.0,85.0,72.0,191.0
4,3,OH,5213,OAJ,"Jacksonville/Camp Lejeune, NC",CLT,1036,1029.0,0.0,0.0,8.0,1037.0,1123.0,36.0,1201,-2.0,0.0,0.0,85.0,90.0,191.0


In [24]:
#data['CRSDepTime'] = df['CRSDepTime'].astype(str).str[:-2].astype(np.int64)
tab_info=pd.DataFrame(data.dtypes).T.rename(index={0:'column type'})
tab_info=tab_info.append(pd.DataFrame(data.isnull().sum()).T.rename(index={0:'null values (nb)'}))
tab_info=tab_info.append(pd.DataFrame(data.isnull().sum()/df.shape[0]*100)
                         .T.rename(index={0:'null values (%)'}))
tab_info




,DayOfWeek,IATA_CODE_Reporting_Airline,Flight_Number_Reporting_Airline,Origin,OriginCityName,Dest,CRSDepTime,DepTime,DepDelayMinutes,DepDel15,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,CRSElapsedTime,ActualElapsedTime,Distance
column type,int64,object,int64,object,object,object,int64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64
null values (nb),0,0,0,0,0,0,0,4226,4227,4227,4367,4367,4595,4595,0,5346,5346,5346,0,5346,0
null values (%),0,0,0,0,0,0,0,0.701466,0.701631,0.701631,0.72487,0.72487,0.762715,0.762715,0,0.887372,0.887372,0.887372,0,0.887372,0


In [34]:
main_airports = data[["Origin","Dest"]].groupby(["Origin"]).count().sort_values(by="Dest",ascending=False)
for i in range(len(data["Origin"])):
    if i in main_airports == False:
        data[Origin]

,Dest
Origin,
JAX,2634
RSW,3153
SNA,3238
SAT,3255
CVG,3808
CLE,3816
IND,4062
CMH,4130
PIT,4183


In [32]:
data[["Dest","Origin"]].groupby(["Dest"]).count().sort_values(by="Origin")

,Origin
Dest,
PPG,8
STC,9
OWB,9
ADK,9
OGD,10
HGR,13
BFM,16
BKG,16
SMX,18
